### Necessary Imports 

In [1]:
import tensorflow as tf
from tensorflow.keras import datasets, layers, models, optimizers
import numpy as np


### Constants 

In [2]:
img_channels = 3
img_rows = 32
img_cols = 32
batch_size = 128
epochs = 20
classes = 10
validation_split = 0.2
optimizer = tf.keras.optimizers.Adam()



### Model Definition

In [25]:
def build(input_shape, classes):
    
    model = models.Sequential()
    
    # Conv2D => Conv2D => Pooling => Dropout
    model.add(layers.Conv2D(32, (3,3), padding='same', 
                           input_shape=X_train.shape[1:],
                           activation='relu'))
    model.add(layers.BatchNormalization())
    model.add(layers.Conv2D(32, (3,3), padding='same',
                           activation='relu'))
    model.add(layers.BatchNormalization())
    model.add(layers.MaxPool2D(pool_size=(2,2)))
    model.add(layers.Dropout(0.25))
    
    
    # Conv2D => Conv2D => Pooling => Dropout
    model.add(layers.Conv2D(64, (3,3), padding='same', 
                           activation='relu'))
    model.add(layers.BatchNormalization())
    model.add(layers.Conv2D(64, (3,3), padding='same',
                           activation='relu'))
    model.add(layers.BatchNormalization())
    model.add(layers.MaxPool2D(pool_size=(2,2)))
    model.add(layers.Dropout(0.25))
    
    
    # Conv2D => Conv2D => Pooling => Dropout
    model.add(layers.Conv2D(128, (3,3), padding='same', 
                           activation='relu'))
    model.add(layers.BatchNormalization())
    model.add(layers.Conv2D(128, (3,3), padding='same',
                           activation='relu'))
    model.add(layers.BatchNormalization())
    model.add(layers.MaxPool2D(pool_size=(2,2)))
    model.add(layers.Dropout(0.25))
    
    
    # Flatten => Dropout => Dense => Dense
    
    model.add(layers.Flatten())
    model.add(layers.Dropout(0.2))
    model.add(layers.Dense(512, activation='relu'))
    model.add(layers.Dense(classes, activation='softmax'))
    
    return model

###  Loading Data

In [26]:
def load_data():
    (X_train, y_train), (X_test, y_test) = datasets.cifar10.load_data()
    X_train = X_train.astype('float32')
    X_test = X_test.astype('float32')
    
    # Reshape
    X_train = X_train.reshape((50000, 32, 32, 3))
    X_test = X_test.reshape((10000, 32, 32, 3))
    
    #normalize
    mean = np.mean(X_train, axis=(0, 1, 2, 3))
    sdv = np.std(X_train, axis=(0, 1, 2, 3))
    X_train = (X_train-mean)/sdv
    X_test = (X_test-mean)/sdv

    y_train = tf.keras.utils.to_categorical(y_train, num_classes=classes)
    y_test = tf.keras.utils.to_categorical(y_test, num_classes=classes)
    
    return X_train, y_train , X_test, y_test

X_train, y_train, X_test, y_test = load_data()

In [28]:
cifar_model = build(X_train.shape[1:], classes)
cifar_model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_9 (Conv2D)            (None, 32, 32, 32)        896       
_________________________________________________________________
batch_normalization_6 (Batch (None, 32, 32, 32)        128       
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 32, 32, 32)        9248      
_________________________________________________________________
batch_normalization_7 (Batch (None, 32, 32, 32)        128       
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 16, 16, 32)        0         
_________________________________________________________________
dropout_4 (Dropout)          (None, 16, 16, 32)        0         
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 16, 16, 64)       

###  Callback

In [29]:
callbacks = [ tf.keras.callbacks.TensorBoard(log_dir='./logs')]


### DataAugmentation 

In [30]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(
            rotation_range=40,
            width_shift_range=0.2,
            height_shift_range=0.2,
            horizontal_flip=True,
            shear_range=0.2)

datagen.fit(X_train)

### Model Compilation 

In [32]:
cifar_model.compile(loss='categorical_crossentropy', optimizer=optimizer,
                   metrics = ['accuracy'])


### Training  

In [33]:
hist = cifar_model.fit_generator(datagen.flow(X_train, y_train,
                                       batch_size= batch_size),
                          epochs = epochs, verbose=1, validation_data=(X_test, y_test)
#save to disk


  ...
    to  
  ['...']
Train for 391 steps, validate on 10000 samples
Epoch 1/20
391/391 [==============================] - 309s 790ms/step - loss: 1.7585 - accuracy: 0.3761 - val_loss: 1.5533 - val_accuracy: 0.4424
Epoch 2/20
391/391 [==============================] - 293s 748ms/step - loss: 1.4088 - accuracy: 0.4892 - val_loss: 1.2644 - val_accuracy: 0.5521
Epoch 3/20
391/391 [==============================] - 292s 747ms/step - loss: 1.2629 - accuracy: 0.5430 - val_loss: 1.3938 - val_accuracy: 0.5465
Epoch 4/20
391/391 [==============================] - 308s 787ms/step - loss: 1.1662 - accuracy: 0.5839 - val_loss: 1.2678 - val_accuracy: 0.5867
Epoch 5/20
391/391 [==============================] - 311s 796ms/step - loss: 1.0872 - accuracy: 0.6104 - val_loss: 1.1046 - val_accuracy: 0.6203
Epoch 6/20
391/391 [==============================] - 302s 772ms/step - loss: 1.0260 - accuracy: 0.6341 - val_loss: 0.9849 - val_accuracy: 0.6634
Epoch 7/20
391/391 [==============================] 

NameError: name 'cifar_model_json' is not defined

In [36]:
model_json = cifar_model.to_json()
with open('model.json', 'w') as json_file:
    json_file.write(model_json)
    
cifar_model.save_weights('model.h5')

